# ETL SINIESTROS VIALES 
# Ciudad Autonoma de Buenos Aires CABA

**Por** 

**Data Analyst**

**GABRIEL GOMEZ RESTREPO**




Para  los Datasets  este  estudio sobre los accidentes viales de la Ciudad Autonoma de Buenos Aires(CABA), Argentina.
recurrimos a consultar  la siguientes links: 

**Para siniestros**

homicidios.xlsx   ( Hechos y Victimas)

https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

**Para poblacion por Comuna** 

PBP_CO1025.xls   (poblacion por comuna  por edad y sexo)

proy_1025_depto_caba.xls   (resumen de PBP_CO1025, poblacion por comuna y por sexo)

https://data.buenosaires.gob.ar/dataset/estructura-poblacion


http://www.indec.mecon.ar/ftp/cuadros/poblacion

**Para Bariios por Comuna** 

comunas.xlsx  (Barrios por comuna)

https://data.buenosaires.gob.ar/dataset/comunas

La revision inicial de los datos que hemos otenido requiere de su estraccion tranformacion y  finalmente Cargar o crear una nueva base de datos usando nombres auque similares para  diferenciarlos con las fuentes originales.
## Librerias 
Las siguientes librerias son las que utilice. 


* **Pandas (import pandas as pd):**
Pandas es una biblioteca de Python fundamental en manejo de DataFrame que proporciona estructuras de datos flexibles y herramientas de análisis. 

* **NumPy (import numpy as np):**
NumPy es una biblioteca fundamental en Python para realizar operaciones numéricas.

* **%load_ext autoreload y %autoreload 2:**
Estos comandos son específicos de los cuadernos Jupyter y se utilizan para recargar automáticamente módulos antes de ejecutar una celda. %load_ext autoreload habilita la recarga automática de módulos, y %autoreload 2 configura la recarga automática para que sea más agresiva y recargue incluso las funciones de los módulos.

* **Warnings (import warnings):**
El módulo warnings proporciona herramientas para controlar las advertencias que emite Python. En este caso, se está configurando para ignorar las advertencias, lo cual puede ser útil para evitar que las advertencias llenen la salida de la consola y distraigan durante la ejecución del código.

In [654]:
import numpy as np
import pandas as pd
from datetime import datetime, time


## Funciones 

**Las funciones nos ahorran tiempo:**
>Esta son las funciones que utilice, generar unas funciones para agilizar nuestras tareas es muy importante evitando muchas acciones repetitivas ademas permiten un proceso mas claro y ordenado a continuacion las relaciono:  

> Funcion N°1A 'detalleA_tipos_dato(df)'

mustra todos los tipos de datos detallados por columna 
 
> Funcion N°1B 'detalleB_tipos_dato(df)'

Muestra solo colummas con dos o mas tipos de datos

>Funcion N°1C 'vistaC_tipo_datos(df, columna)'

Muetra 5 filas por cada tipo de dato de una columna

>Funcion N°2 'ver_nulos(df)'

Muestra un cuadro con las columnas que tienen nulos y su %

Y muestra un segudo cuadro con todas las columnas de nulos, no nulos y su %

>Funcion N°3 'ver_nulos_por_colref(df, colref)'**

Muestra  solo filas de todo el DataFrame o Grupo de colomnas previamente elegidas,

donde una columna que que usemos de referencia tenga nulos   

>Funcion N°4 'def conversor_tiempo(df, columna, valor)'

Convierte a un valor de  dato de  tipo tiempo (datetime.time)

>Funcion N°5A 'imputa_valor_por_moda(df, columna)'

LLena celdas nulas con la moda o valor mas frecuente 

>Funcion N°5B1 'Imputacion_por_colref(df, colref, col_imputadora)'

Llena datos nulos de una columna con los datos de otra
*Imputa los valores nulos de la columna de referencia (colref) utilizando los valores de otra columna (col_imputadora).*

*Además, muestra las filas donde la columna de referencia tiene valores nulos antes de la imputación.*

>Funcion N°5B2 'Imputacion_AreplaceB_colref(df, colref, col_replace, a, b):'

*Imputa a por b dcon base a los nulos de una columan  de referencia (colref) .*

>Funcion N°5C 'imputa_edad_media_por_sexo(df)'


*Llena los valores 'SD' con la  `media` de columna  'Edad'   considerando el genero  de la columna 'Sexo' 


# INICIIO FUNCIONES

**Funcion N°1A 'detalleB_tipos_dato(df)'**

>APLICACION

*Esta funcion selecciona los tipos para cada columna  por el criterio de objeto* 

*Cuando una columna tiene un solo tipo, lo muestra* 

*Cuando* una columna tiene un solo tipo, y tiene nulos  lo muestra como *objetos* 

*Cuando una columna tiene varios tipos de datos  los clasifica como *objetos* 

*Con* esta fucion vemos en detalle  cales son esos tipos de datos clasificados como objetos* 



In [655]:

#Esta función recorre las columnas de un DataFrame y muestra el tipo de dato de cada columna,
# junto con los tipos de datos únicos si la columna es de tipo objeto.
def detalleA_tipos_datos(df):
   
    for columna, tipo_dato in df.dtypes.items():  # Recorre cada par clave-valor en df.dtypes.items()
        if pd.api.types.is_object_dtype(tipo_dato):  # Si el tipo de dato actual es un tipo de objeto
            tipos_unicos = df[columna].apply(type).unique()  # Obtiene los tipos de datos únicos en la columna
            print(f"{columna}: {tipos_unicos}")  # Imprime  la columna que tiene un solo tipo de dato
        else:  # Si el tipo de dato actual no es un tipo de objeto
            print(f"{columna}: {tipo_dato}")  # Imprime  la columna y todos sus tipos de datos


**Funcion N°1B 'detalleC_tipos_dato(df)'**

>APLICACION

*Esta funcion selecciona solo las columnas con solo objetos, osea mas de un tipo de datos*  

*Cuando una columna tiene un solo tipo, y tiene nulos los clasifica tambien como objetos* 

*Cuando una columna tiene varios tipos de datos los clasifica como objetos* 


In [656]:

def detalleB_tipos_datos(df):
   
    for columna, tipo_dato in df.dtypes.items():  # Recorre cada par clave-valor en df.dtypes.items()
        if pd.api.types.is_object_dtype(tipo_dato):  # Si el tipo de dato actual es un tipo de objeto
            tipos_unicos = df[columna].apply(type).unique()  # Obtiene los tipos de datos únicos en la columna
            print(f"{columna}: {tipos_unicos}")  # Imprime  la columna que tiene un solo tipo de dato
        else:  # Si el tipo de dato actual no es un tipo de objeto
            print(f" ")  # Imprime  la columna y todos sus tipos de datos


**Funcion N°1C 'vistaC_tipo_datos(df, columna)'**

>APLICACION

*Nos permite ver  5 filas por cada tipo de datos de una columna en un  DataFrame* 

*con el ojeto de tener mas claridad antes de desidir trasformarlos*

In [657]:


def vistaC_tipo_datos(df, columna):
    # Crear diccionario para almacenar las muestras de filas por tipos de datos
    muestras_por_tipos = {}

    print(f"\n\nColumna: {columna}")
    # Obtener los tipos de datos únicos en la columna
    tipos_datos = df[columna].apply(lambda x: type(x) if x is not None else None).unique()
    
    for tipo in tipos_datos:
        try:
            # Filtrar las filas por tipo de dato
            if tipo is None:
                filas_tipo = df[df[columna].isnull()]
            else:
                filas_tipo = df[df[columna].apply(lambda x: isinstance(x, tipo))]                            
            # Verificar si hay filas del tipo encontrado
            if not filas_tipo.empty:
                # Guardar el tipo de dato y las primeras 5 filas en el diccionario
                muestras_por_tipos[tipo] = filas_tipo[[columna]].head(5)
        except Exception as e:
            print(f"Error procesando tipo {tipo} en columna {columna}: {e}")

    return muestras_por_tipos




**Funcion N°2 'ver_nulos(df)'**

>APLICACION

*El proposito esta funvion es mostrar con relevacia las columnas que tienen nulos en  un DataFrame mortrandonos* 

*el porcentaje,  adicionalmete nos muestra un cuadro de resumen total de null y notnull* 





In [658]:

def ver_nulos(df):
    # Crear diccionario para almacenar información sobre los nulos y no nulos
    dic = {"campo": [], "%notnull": [], "%null": [], "notnull": [], "nulos": []}      
    for columna in df.columns:  # Iterar sobre todos los campos del DataFrame df
        # Calcular el porcentaje de No nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        # Llenar el diccionario
        dic["campo"].append(columna)
        dic["%notnull"].append(round(porcentaje_no_nulos, 2))      
        dic["%null"].append(round(100 - porcentaje_no_nulos, 2))
        dic["notnull"].append(df[columna].notnull().sum())
        dic["nulos"].append(df[columna].isnull().sum())             

    # Crear otro DataFrame con la información nueva 
    df_nulos = pd.DataFrame(dic)

    # Calcular los nulos en el DataFrame      
    total_nulos = df.isnull().sum().sum()  

    # Imprimir el total de valores nulos
    print(f'TOTAL NULOS DataFrame: {total_nulos}')  

    # Mostrar los campos o columnas con valores nulos y %
    campos_con_nulos = df_nulos[df_nulos["nulos"] > 0][["campo", "nulos", "%null"]].sort_values(by="%null", ascending=False)
    if not campos_con_nulos.empty:
        print("\nCAMPOS QUE TIENEN VALORES NULOS Y SU %:")       
        print(campos_con_nulos)

    # Retornar el DataFrame con la información de cada columna
    return df_nulos  




**Funcion N°3 'ver_nulos_por_colref(df, colref)'**

>APLICACION

*Muestra un DataFrame con las filas donde la columna referencia elegida (colref),  tiene valores nulos*

*Esto permite  filtarar todas las columnas con el mismos'Id' esto es muy util para asociar que pasa con las celdas correspondientes.*

*por ejemplo, ver  que los datos de otra columna podran ser utiles para llenar los nulos, o  considerar su importacia para par ver*

*como una columna con pocoas datos, su eliminacion no, podria afentar las otras  .*

In [659]:
#  Muestra un DataFrame con las filas donde la columna referencia elegida (colref), elegida   tiene valores nulos.
#  EJEMPLO para ver nulos 'Hora' relacionada con otras columnas : ver_nulos_por_colref(df_hechos, 'Hora')[['Id', 'Hora', 'Hora entera', 'Fecha']]
# asi limitamos el numero de columnas y solo llamamos las relacionadas segun nuestro criterio 

def ver_nulos_por_colref(df, colref):
          
    if colref not in df.columns:
        raise ValueError(f"La columna '{colref}' no existe en el DataFrame")
        
    nulos = df[df[colref].isnull()]
    
    return nulos


**Funcion N°4 'def conversor_tiempo(df, columna, valor)'**

>APLICACION

*Convierte a un valor de  dato de  tipo tiempo (datetime.time)*
                
*valor: Valor a ser convertido. Puede ser str, datetime.datetime o datetime.time.*

In [660]:


def conversor_tiempo(df, columna, valor):
   
    def convertir_tiempo(valor):
       
        #Convierte un valor de tipo tiempo a datetime.time si es posible.
                
        #valor: Valor a ser convertido. Puede ser str, datetime.datetime o datetime.time.
               
        
        if isinstance(valor, str):
            try:
                # Intenta convertir el string a datetime.time
                return datetime.strptime(valor, '%H:%M:%S').time()
            except ValueError:
                try:
                    return datetime.strptime(valor, '%H:%M').time()
                except ValueError:
                    return None
        elif isinstance(valor, datetime):
            # Si es datetime.datetime, devuelve datetime.time truncando los microsegundos
            return valor.replace(microsecond=0).time()
        elif isinstance(valor, time):
            # Si es datetime.time, no es necesario hacer nada
            return valor.replace(microsecond=0)
        else:
            return None
    
    # Asigna el valor convertido a la columna especificada en el DataFrame
    df[columna] = df[columna].apply(convertir_tiempo)
    
    return df


**Funcion N°5A 'imputa_valor_por_moda(df, columna)'**

>APLICACION

*Esta función reemplaza los valores "SD" con NaN en la columna especificada,*

*luego calcula la moda (el valor más frecuente) en esa columna y utiliza ese valor*

*para imputar los valores faltantes (NaN).*

In [661]:
def imputa_valor_por_moda(df, columna):

    # Se reemplaza "SD" con NaN en la columna
    df[columna] = df[columna].replace('SD', pd.NA)

    # Se calcula el valor más frecuente en la columna
    moda = df[columna].mode().iloc[0]
    print(f'El valor mas frecuente es: {moda}')

    # Se imputan los valores NaN con el valor más frecuente
    df[columna].fillna(moda, inplace=True)

**Funcion N°5B1 'Imputacion_por_colref(df, colref, col_imputadora)'**

>APLICACION

*Imputa los valores nulos de la columna de referencia (colref) utilizando los valores de otra columna (col_imputadora).*

*Además, muestra las filas donde la columna de referencia tiene valores nulos antes de la imputación.*

In [662]:
import pandas as pd

def Imputacion_por_colref(df, colref, col_imputadora):
      
    #df (pd.DataFrame): DataFrame donde se realizará la imputación.
    #colref (str): Nombre de la columna de referencia con valores nulos a imputar.
    #col_imputadora (str): Nombre de la columna cuyo valor se usará para imputar.
       
    # Verificar que las columnas existen en el DataFrame
    if colref not in df.columns:
        raise ValueError(f"La columna '{colref}' no existe en el DataFrame")
    if col_imputadora not in df.columns:
        raise ValueError(f"La columna '{col_imputadora}' no existe en el DataFrame")

    # Mostrar las filas donde la columna de referencia tiene valores nulos
    nulos = df[df[colref].isnull()]
    print(f"Filas con valores nulos en la columna '{colref}':")
    print(nulos)
    
    # Imputar los valores nulos de colref usando los valores de col_imputadora
    df[colref] = df.apply(
        lambda row: row[col_imputadora] if pd.isnull(row[colref]) else row[colref], axis=1
    )
    
    return df

# Ejemplo de uso 
# df = pd.DataFrame({'Hora': [None, '15:45:00', None], 'Hora_entera': ['12:30:00', '15:45:00', '08:00:00'], 'Fecha': ['2022-01-01', '2022-01-02', '2022-01-03']})
# df = Imputacion_por_colref(df, 'Hora', 'Hora_entera')
# print(df)


**Funcion N°5B2 'Imputacion_AreplaceB_colref(df, colref, col_replace, a, b):'**

>APLICACION

*Imputa a por b dcon base a los nulos de una columan  de referencia (colref) .*



In [663]:
import pandas as pd

def Imputacion_AreplaceB_colref(df, colref, col_replace, a, b):
    """
    Función para imputar valores en una columna basada en los valores nulos de una columna de referencia.
    
    Args:
        df (pd.DataFrame): DataFrame donde se realizará la imputación.
        colref (str): Nombre de la columna de referencia con valores nulos a considerar.
        col_replace (str): Nombre de la columna donde se realizará la imputación.
        a (var): Valor actual en col_replace que se reemplazará.
        b (var): Nuevo valor para reemplazar 'a' en col_replace.
    
    Returns:
        pd.DataFrame: DataFrame con los valores imputados.
    """
    
    # Verificar que las columnas existen en el DataFrame
    if colref not in df.columns:
        raise ValueError(f"La columna '{colref}' no existe en el DataFrame")
    if col_replace not in df.columns:
        raise ValueError(f"La columna '{col_replace}' no existe en el DataFrame")
    
    # Mostrar las filas donde la columna de referencia tiene valores nulos
    nulos = df[df[colref].isnull()]
    print(f"Filas con valores nulos en la columna '{colref}':")
    print(nulos)
    
    # Imputar los valores en col_replace basados en los nulos en colref
    df.loc[df[colref].isnull() & (df[col_replace] == a), col_replace] = b
    
    return df

# Ejemplo de uso 
# df = pd.DataFrame({'Hora': [None, '15:45:00', None], 'Hora_entera': ['12:30:00', '15:45:00', '08:00:00'], 'Fecha': ['2022-01-01', '2022-01-02', '2022-01-03']})
# df = Imputacion_AreplaceB_colref(df, 'Hora', 'Hora_entera', '12:30:00', '00:00:00')
# print(df)


**Funcion N°5C 'imputa_edad_media_por_sexo(df)'**

>APLICACION

*Esta función reemplaza los valores "SD" con NaN en la columna 'Edad', calcula la `media` para conseguir la edad promedio*

*para cada grupo de género (Femenino y Masculino), imprime los promedios calculados y*

*luego llena los valores faltantes en la columna 'Edad' utilizando el promedio correspondiente*

*al género al que pertenece cada fila en el DataFrame.*

In [664]:
def imputa_edad_media_por_sexo(df):

    # Se reemplaza "SD" con NaN en la columna 'edad'
    df['Edad'] = df['Edad'].replace('SD', pd.NA)

    # Se calcula el promedio de edad para cada grupo de género
    promedio_por_genero = df.groupby('Sexo')['Edad'].mean()
    print(f'La edad promedio de Femenino es {round(promedio_por_genero["FEMENINO"])} y de Masculino es {round(promedio_por_genero["MASCULINO"])}')

    # Se llenan los valores NaN en la columna 'edad' utilizando el promedio correspondiente al género
    df['Edad'] = df.apply(lambda row: promedio_por_genero[row['Sexo']] if pd.isna(row['Edad']) else row['Edad'], axis=1)
    # Lo convierte a entero
    df['Edad'] = df['Edad'].astype(int)

**Funcion N°6 'buscar_puntos(df)'**

>APLICACION

*Función para buscar celdas que contienen un solo punto (".") en el DataFrame*

In [665]:

 #Función para buscar celdas que contienen un solo punto (".") en el DataFrame y mostrar las filas y columnas donde se encuentran.  
 
 # ejenplo para buscar si se quiere en solo  algunas columnas
 # buscar_puntos(df_hechos)[['Lugar del hecho', 'Comuna', 'XY (CABA)', 'Pos x',  'Pos y']]
 
def buscar_puntos(df): 
      
       
    # Convertir todas las columnas a string para facilitar la búsqueda de "."
    df_str = df.astype(str)
    
    # Crear una máscara booleana donde se encuentra el "."
    mask = df_str.applymap(lambda x: x == '.')
    
    # Filtrar las filas que contienen al menos un punto
    filas_con_puntos = df[mask.any(axis=1)]
    
    print("Filas y columnas que contienen un solo punto ('.'):")
    for row in mask.index:
        for col in mask.columns:
            if mask.at[row, col]:
                print(f"Fila: {row}, Columna: {col}")
    
    return filas_con_puntos 


**Funcion N°7 'eliminar_filas_con_puntos_colref(df, colref)'**

>APLICACION

*Función para eliminar las filas  de todo el   DataFrame,  basado   en columna de referencia `colref`que contienen  un solo punto (".")*

In [666]:


def eliminar_filas_con_puntos_colref(df, colref):
  
    # Función para eliminar las filas que contienen un solo punto (".") en una columna especificada del DataFrame.
       
    
    # Obtener los índices de las filas que contienen un solo punto en la columna de referencia
    indices_filtrados = df.index[df[colref].astype(str) == '.'].tolist()
    
    # Eliminar las filas que contienen un solo punto en la columna de referencia del DataFrame original
    df.drop(df.index[indices_filtrados], inplace=True)
    
    # Retornar la lista de índices de las filas eliminadas
    return indices_filtrados


# CREACION DATAFRAME

**EXPLICACION**
>De homicidios.xlsx sacamos  df_hechos 

>Extraemos  de Datasets/Fuente el archivo homicidios.xlsx, lo leemos y creamos el DataFrame  'df_hechos 



In [667]:
df_hechos = pd.read_excel('./Datasets/Fuente/homicidios.xlsx', sheet_name='HECHOS')

**Miramos aspectos la informacion general del DataFrame y concluimos a primera vista que:** 

*Los  titulos de  columnas  algunas estan en mayusculas y otras en minusculas, hay nombres que podrian ser mas fasiles de entender por ejemplo seria mejos 'Año' que AAAA*  

*Tenemos la mayoria de las Columnas sin nulos (696) pero  TIPO_DE_CALLE, Calle, Altura, Cruce  si tienen presencia de nulos*

*Y tenemos los siguentes  tipos de datos  dtypes: datetime64[ns](1), float64(1), int64(5), object(14)*


In [668]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

## ESTANDARIZAR TITULOS

#### EXPLICACION

* MAYUSCULA   a  May-Minusclula

* AAAA   por  Año

 *Modificamos  la primera letra en mayusculas y las demas en minuscula*

 *Cambiamoa los titulos de algunas columnas  como para mejorar la fasilidad de comprension* 
 
 

In [669]:
df_hechos.columns = df_hechos.columns.str.replace('_', ' ') # guiones por espacios
df_hechos.columns = [x.capitalize() for x in df_hechos.columns] #  mayúscula letra 1 y resto a minusculas
# Cambiamos algunos titulos 
df_hechos = df_hechos.rename(columns={'N victimas': 'Cantidad víctimas', 'Aaaa':'Año', 'Mm':'Mes', 'Dd':'Día',
                                      'Hh':'Hora entera', 'Xy (caba)':'XY (CABA)', 'Victima': 'Víctima'})

df_hechos.columns # oservando como quedan los titulares de las columnas

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora entera', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura',
       'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x',
       'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

# NORMALIZAR TIPOS DE DATOS 

**EXPLICACION**
>Ya hemos notado que para este analisis no podriamos dejar df_hechos sin acomodar el tipo de datos a nuestro requerimientos  

>No, no es obligatorio en un DataFrame de pandas contar con un solo tipo de dato por columna. Pandas es una herramienta flexible que permite almacenar diferentes tipos de datos en cada columna de un DataFrame. 

Sin embargo, **en general, se recomienda tener un único tipo de dato por columna para facilitar el análisis y la manipulación de datos**. Esto se debe a que:

- **Simplicidad:** Al tener un solo tipo de dato por columna, las operaciones matemáticas y de análisis se vuelven más sencillas y directas. Pandas no necesita realizar conversiones de tipo de dato entre elementos de la misma columna, lo que mejora el rendimiento y la legibilidad del código.

- **Consistencia:** Un DataFrame con tipos de datos consistentes en cada columna facilita la comprensión de la estructura del mismo y la interpretación de los resultados de análisis. 

- **Herramientas de análisis:** Muchas herramientas y funciones de pandas están diseñadas específicamente para trabajar con un solo tipo de dato por columna. Si se mezclan tipos de datos, es posible que estas herramientas no funcionen correctamente o generen resultados inesperados.

**Excepciones:**

- **Columnas de texto:** En algunos casos, puede ser necesario tener una columna de texto (tipo `object`) para almacenar datos no numéricos, como nombres, descripciones o categorías.

- **Columnas con datos mixtos:** En situaciones específicas, puede ser útil tener una columna con datos mixtos, como números y cadenas de texto. Sin embargo, esto debe manejarse con cuidado y se recomienda evaluar si la estructura del DataFrame es la más adecuada para el análisis que se pretende realizar.

**En resumen:**

Si bien no es obligatorio tener un solo tipo de dato por columna en un DataFrame, se recomienda encarecidamente hacerlo para facilitar el análisis, la manipulación y la comprensión de los datos. Si necesita almacenar datos de diferentes tipos, considere utilizar columnas separadas o estructuras de datos alternativas, como Series de pandas o DataFrames anidados.


**VISUALIZAMOS LOS TIPOS DE DATOS DEL DATAFRAME**

>dt_hechos

*usamos la Funcion N°1A 'detalleB_tipo_dato(df)'*

*que nos permite ver todos los tipos de datos por columna*

In [670]:
detalleB_tipos_datos(df_hechos)


Id: [<class 'str'>]
 
 
 
 
 
Hora: [<class 'datetime.time'> <class 'str'> <class 'datetime.datetime'>]
Hora entera: [<class 'int'> <class 'str'>]
Lugar del hecho: [<class 'str'>]
Tipo de calle: [<class 'str'>]
Calle: [<class 'str'> <class 'float'>]
 
Cruce: [<class 'str'> <class 'float'>]
Dirección normalizada: [<class 'str'> <class 'float'>]
 
XY (CABA): [<class 'str'>]
Pos x: [<class 'str'>]
Pos y: [<class 'str'>]
Participantes: [<class 'str'>]
Víctima: [<class 'str'>]
Acusado: [<class 'str'>]


### SOLO COLUMAS CON MULTIPLES TIPOS DE DATOS

 *Elegimos solo las columnas con datos multiples o tipo objeto y para eso usamos la Funcion N°1B 'detalleB_tipo_dato(df)'*



In [671]:
detalleB_tipos_datos(df_hechos)

Id: [<class 'str'>]
 
 
 
 
 
Hora: [<class 'datetime.time'> <class 'str'> <class 'datetime.datetime'>]
Hora entera: [<class 'int'> <class 'str'>]
Lugar del hecho: [<class 'str'>]
Tipo de calle: [<class 'str'>]
Calle: [<class 'str'> <class 'float'>]
 
Cruce: [<class 'str'> <class 'float'>]
Dirección normalizada: [<class 'str'> <class 'float'>]
 
XY (CABA): [<class 'str'>]
Pos x: [<class 'str'>]
Pos y: [<class 'str'>]
Participantes: [<class 'str'>]
Víctima: [<class 'str'>]
Acusado: [<class 'str'>]


**ANALISAMOS EL RESULTADO**


>`Hora` Esta colunna es la que tiene mas tipos de datos(3): 

*(datetime.time , datetime.datetime y str)*
 

>`'Calle', 'Cruce' y 'Dirección normalizada'`  tienen tipos de datos:

*(str y float)*

> `'Hora entera'`  tambien tiene dos tipos: 

*(str y int)*

>`'Id', 'Lugar del hecho', 'Tipo de calle',  'XY (CABA)', 'Pos x', 'Pos y', 'Participantes'`,

> `'Víctima' y 'Acusado'` tienen nulos y un solo tipo de dato:

*(null y str)* 

  

## Hora

*Empecemis por corregir  la columna  `Hora`*

*Veamos las cantidades por tipo*


In [672]:

df_hechos['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

**VISTAZO**

 df_hechos['Hora']

 *Para formarnos una idea mas clara hacemos un breve pantallazo usando la  Funcion N°1D 'vista_tipo_datos(df, columna)'*

 >Asi podemos ver la forma de estos tres tipos de Datos de la columna `Hora` y es posible que tambien fifuren algunos nulos, aqui tambien se vera

### Funcion N°1C 'vista_tipo_datos(df, columna)'

In [673]:
vistaC_tipo_datos(df_hechos,'Hora')



Columna: Hora


{datetime.time:        Hora
 0  04:00:00
 1  01:15:00
 2  07:00:00
 3  00:00:00
 4  05:20:00,
 str:          Hora
 275  22:50:00
 276  16:40:00
 277  10:12:00
 278  09:40:00
 279  22:30:00,
 datetime.datetime:                     Hora
 439  1900-01-02 00:00:00
 441  1900-01-14 07:12:00
 496  1900-01-04 18:40:00}

**CONVERSION TIEMPO**

*Unificamos el tipo de dato ('datetime.time') para todas las celdas de la columna `Hora`*

*usando la funcion Funcion N°4 'def conversor_tiempo(df, columna, valor)'*

In [674]:
conversor_tiempo(df_hechos, 'Hora' , datetime.time)

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


**vemos la columna 'Hora'con tipos de datos (condatetime.time)**

In [675]:
df_hechos['Hora']

0      04:00:00
1      01:15:00
2      07:00:00
3      00:00:00
4      05:20:00
         ...   
691    17:10:00
692    01:10:00
693    00:43:00
694    10:30:00
695    06:10:00
Name: Hora, Length: 696, dtype: object

**Nos muestra que ha quedado un null y por eso no tiene dato** 

In [676]:
df_hechos['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>    695
<class 'NoneType'>           1
Name: count, dtype: int64

In [677]:
vistaC_tipo_datos(df_hechos, 'Hora')



Columna: Hora


{datetime.time:        Hora
 0  04:00:00
 1  01:15:00
 2  07:00:00
 3  00:00:00
 4  05:20:00,
 None:      Hora
 518  None}

In [678]:
ver_nulos_por_colref(df_hechos, 'Hora')[['Id', 'Hora', 'Hora entera', 'Fecha']]

,Id,Hora,Hora entera,Fecha
518,2019-0103,None,SD,2019-12-18


**IMPUTANDO**

*imputamos la unica celda vacia usando  la funcion Funcion N°5A`imputa_valor_por_moda(df_hechos, 'Hora')` para garantizar que los tipos de datos*

*queden bajo nuestro criterio y no dejar nada al azar, la opcion de eliminar la fila la considere pero prefiero no perder el resto de informacion*

In [679]:
imputa_valor_por_moda(df_hechos, 'Hora')

El valor mas frecuente es: 09:00:00


## Hora entera

*Sabemos que tiene dos tipoa de datos (str y int)*

*Veamos las cantidades por tipo*


`'Hora entera'`  tambien tiene dos tipos: 

*(str y int)*

>Podemos ver que solo hay un 'str' vamos a imputar tambien por moda pero vamos a ver detalles 

In [680]:
df_hechos['Hora entera'].apply(type).value_counts()

Hora entera
<class 'int'>    695
<class 'str'>      1
Name: count, dtype: int64

In [681]:
vistaC_tipo_datos(df_hechos, 'Hora entera')



Columna: Hora entera


{int:   Hora entera
 0           4
 1           1
 2           7
 3           0
 4           5,
 str:     Hora entera
 518          SD}

**IMPUTANDO**

*imputamos la unica celda 'str' usando  la funcion Funcion N°5A`imputa_valor_por_moda(df_hechos, 'Hora entera')` para garantizar que los tipos de datos*



In [682]:
imputa_valor_por_moda(df_hechos, 'Hora entera')

El valor mas frecuente es: 7


*Comprovando la imputacion*

In [683]:
vistaC_tipo_datos(df_hechos, 'Hora entera')



Columna: Hora entera


{int:    Hora entera
 0            4
 1            1
 2            7
 3            0
 4            5}

##  Cruce, Dirección normalizada y Calle

*La situacion de estas 3 columnas  con tipo de datos (str y float) vamos  a revisarlas teniendo en cuenta los siguiente:
1. revisar cual tipo de  que debe predominar 
2. ver que relacion tiene la ausencua de datos (nulos ),  considerar los datos de otras columnas que tambien estan
 asociados al tema de ubicacion territorial 




1. **PREDOMINIO TIPO DE DATO**

*Definitivamente `str` es el tipo de dato a  elegir por la presencia en esencia de cadenas de palabras como podemos ver* 

In [684]:
df_hechos[[ 'Calle', 'Cruce', 'Dirección normalizada']]

,Calle,Cruce,Dirección normalizada
0,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034
3,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."
...,...,...,...
691,RIESTRA AV.,MOM,RIESTRA AV. y MOM
692,"DELLEPIANE, LUIS, TTE. GRAL.",LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV."
693,GAONA AV.,TERRADA,GAONA AV. y TERRADA
694,"PERON, EVA AV.",NaN,"PERON, EVA AV. 4071"


2. **RELACION DATOS NULOS ENTRE COLUMNAS**

*usamos la funcion N°2  `ver_nulos(df_hechos)` para ver los nulos con  procentaje para justificar las columnas a comparar  y asi usar la funcion* 

*N°3 `ver_nulos_por_colref(df_hechos, colref)[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]`*

*Despues de mirar la informacion resultante y tratando de ponerme en el papel de las autoridades en la tarea de registrar el accidente  me imagino que.*

>CONCLUSIONES:

  1. En el momento de registrar el accidente no podria elegirse Cruce y Altura simultaneamente, el porcentage muy alto se de nulos en Altura de debe  a  que  los accidentes suceden principalmente en los  Cruces.

  2. los acidentes en Autopistas y avenidas presentan tramos muy largos entre Cruce y Cruce y no existen placas de propiedades que permitan mostrar un numero como si secede en el caso de `Altura` y esa es la razon para que en esta situacion solo quede registrada en la columna `Lugar del hecho`. por que en 8 casos no figura en la columna  `Dirección normalizada`

  3. Por lo tanto procedere a  imputar  Los nulos de la columna `Altura` que coinciden con los de la columna  `Dirección normalizada`se remplazaran  con 
   0.10(float), interpretese esto  como: no sucedio a aqui fue en  `Autopista`. Todos los demas nulos serian llenados con 0.00(float), interpretese como  sucedio en  `Cruce`. Nota : seria un gran error eliminar la columna `Altura`  al tener muchos  nulos (81.47%), ya que los valores del 18,53% no dice que suceden muchos menos accidentes entre esquina y esquina  
  
  4. imputar los nulos  la columna `Dirección normalizada` con su correspondientes de  la columna `Lugar del hecho`

  4. Los nulos de las columnas   `Cruce` y `Calle` los imputare con el str `SD` SD (sin dato)

  

In [685]:
ver_nulos(df_hechos)

TOTAL NULOS DataFrame: 747

CAMPOS QUE TIENEN VALORES NULOS Y SU %:
                    campo  nulos  %null
11                 Altura    567  81.47
12                  Cruce    171  24.57
13  Dirección normalizada      8   1.15
10                  Calle      1   0.14


,campo,%notnull,%null,notnull,nulos
0,Id,100.00,0.00,696,0
1,Cantidad víctimas,100.00,0.00,696,0
2,Fecha,100.00,0.00,696,0
3,Año,100.00,0.00,696,0
4,Mes,100.00,0.00,696,0
5,Día,100.00,0.00,696,0
6,Hora,100.00,0.00,696,0
7,Hora entera,100.00,0.00,696,0
8,Lugar del hecho,100.00,0.00,696,0
9,Tipo de calle,100.00,0.00,696,0


>Utilizamos la funcion N°3 `ver_nulos_por_colref(df_hechos, colref)` y Comparamos cada una de estas columnas involucradas 

In [686]:
ver_nulos_por_colref(df_hechos, 'Cruce')[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]

,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada
2,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034
9,AV ENTRE RIOS 1366,AVENIDA,ENTRE RIOS AV.,1366.0,NaN,ENTRE RIOS AV. 1366
14,SUIPACHA 156,CALLE,SUIPACHA,156.0,NaN,SUIPACHA 156
33,LIMA 1483,CALLE,LIMA,1483.0,NaN,LIMA 1483
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI
...,...,...,...,...,...,...
680,AU 25 DE MAYO KM7 (ALTURA PASCO),AUTOPISTA,AUTOPISTA 25 DE MAYO,NaN,NaN,AUTOPISTA 25 DE MAYO y PASCO
685,ESMERALDA 1359,CALLE,ESMERALDA,1359.0,NaN,ESMERALDA 1359
687,AV. GAONA 3655,AVENIDA,GAONA AV.,3655.0,NaN,GAONA AV. 3655
694,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.",4071.0,NaN,"PERON, EVA AV. 4071"


In [687]:
ver_nulos_por_colref(df_hechos, 'Altura')[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]

,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada
0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
3,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."
5,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,27 DE FEBRERO AV.,NaN,ESCALADA AV.,27 DE FEBRERO AV. y ESCALADA AV.
...,...,...,...,...,...,...
689,BAIGORRIA Y VICTOR HUGO,CALLE,BAIGORRIA,NaN,"HUGO, VICTOR","BAIGORRIA y HUGO, VICTOR"
690,AV. RIVADAVIA Y AV. PUEYRREDON,AVENIDA,RIVADAVIA AV.,NaN,PUEYRREDON AV.,RIVADAVIA AV. y PUEYRREDON AV.
691,AV. RIESTRA Y MOM,AVENIDA,RIESTRA AV.,NaN,MOM,RIESTRA AV. y MOM
692,AU DELLEPIANE Y LACARRA,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV."


In [688]:
ver_nulos_por_colref(df_hechos, 'Dirección normalizada')[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]

,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada
38,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN
119,SD,CALLE,NaN,NaN,NaN,NaN
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN,NaN,NaN
181,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN,NaN,NaN
313,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN
546,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN
621,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN


In [689]:
ver_nulos_por_colref(df_hechos, 'Calle')[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]

,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada
119,SD,CALLE,NaN,NaN,NaN,NaN


>IMPUTACION

- Funcion N°5B2 'Imputacion_AreplaceB_colref(df, colref, col_replace, a, b)'

- Imputa `a`= valores de altura cuando,  'Dirección normalizada' es nulo  por `b`= 0.10 

In [690]:
Imputacion_AreplaceB_colref(df_hechos, 'Dirección normalizada', 'Altura', np.nan, 0.10)

Filas con valores nulos en la columna 'Dirección normalizada':
            Id  Cantidad víctimas      Fecha   Año  Mes  Día      Hora  \
38   2016-0052                  1 2016-04-20  2016    4   20  20:00:00   
106  2016-0136                  1 2016-10-25  2016   10   25  00:00:00   
119  2016-0151                  1 2016-11-18  2016   11   18  20:35:00   
180  2017-0050                  2 2017-04-28  2017    4   28  11:08:08   
181  2017-0051                  1 2017-05-01  2017    5    1  03:47:47   
313  2018-0039                  1 2018-04-21  2018    4   21  22:15:00   
546  2020-0026                  1 2020-05-17  2020    5   17  06:40:00   
621  2021-0023                  1 2021-03-01  2021    3    1  09:20:00   

     Hora entera                          Lugar del hecho Tipo de calle  ...  \
38            20               AUTOPISTA LUGONES PK 10000     AUTOPISTA  ...   
106            0         AU BUENOS AIRES - LA PLATA KM. 4     AUTOPISTA  ...   
119           20              

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


>IMPUTACION 

-  'Dirección normalizada' con 'Lugar del hecho' usamos Funcion N°5B1 'Imputacion_por_colref(df, colref, col_imputadora)'

In [691]:
Imputacion_por_colref(df_hechos, 'Dirección normalizada', 'Lugar del hecho')

Filas con valores nulos en la columna 'Dirección normalizada':
            Id  Cantidad víctimas      Fecha   Año  Mes  Día      Hora  \
38   2016-0052                  1 2016-04-20  2016    4   20  20:00:00   
106  2016-0136                  1 2016-10-25  2016   10   25  00:00:00   
119  2016-0151                  1 2016-11-18  2016   11   18  20:35:00   
180  2017-0050                  2 2017-04-28  2017    4   28  11:08:08   
181  2017-0051                  1 2017-05-01  2017    5    1  03:47:47   
313  2018-0039                  1 2018-04-21  2018    4   21  22:15:00   
546  2020-0026                  1 2020-05-17  2020    5   17  06:40:00   
621  2021-0023                  1 2021-03-01  2021    3    1  09:20:00   

     Hora entera                          Lugar del hecho Tipo de calle  ...  \
38            20               AUTOPISTA LUGONES PK 10000     AUTOPISTA  ...   
106            0         AU BUENOS AIRES - LA PLATA KM. 4     AUTOPISTA  ...   
119           20              

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [692]:
vistaC_tipo_datos(df_hechos, 'Dirección normalizada')



Columna: Dirección normalizada


{str:                                Dirección normalizada
 0  PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...
 1               PAZ, GRAL. AV. y DE LOS CORRALES AV.
 2                                ENTRE RIOS AV. 2034
 3          LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.
 4            SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.}

>IMPUTACION 

- 'Calle', 'Altura', 'Cruce'  con 'SD'

In [693]:
# Se rellenan nulos con Falta info
df_hechos[ 'Cruce'].fillna("SD", inplace=True)

In [694]:
# Se rellenan nulos con Falta info
df_hechos[ 'Altura'].fillna(0.00, inplace=True)

In [695]:
# Se rellenan nulos con Falta info
df_hechos[ 'Calle'].fillna("SD", inplace=True)

In [696]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     696 non-null    object        
 1   Cantidad víctimas      696 non-null    int64         
 2   Fecha                  696 non-null    datetime64[ns]
 3   Año                    696 non-null    int64         
 4   Mes                    696 non-null    int64         
 5   Día                    696 non-null    int64         
 6   Hora                   696 non-null    object        
 7   Hora entera            696 non-null    int64         
 8   Lugar del hecho        696 non-null    object        
 9   Tipo de calle          696 non-null    object        
 10  Calle                  696 non-null    object        
 11  Altura                 696 non-null    float64       
 12  Cruce                  696 non-null    object        
 13  Direc

In [697]:
detalleB_tipos_datos(df_hechos)

Id: [<class 'str'>]
 
 
 
 
 
Hora: [<class 'datetime.time'>]
 
Lugar del hecho: [<class 'str'>]
Tipo de calle: [<class 'str'>]
Calle: [<class 'str'>]
 
Cruce: [<class 'str'>]
Dirección normalizada: [<class 'str'>]
 
XY (CABA): [<class 'str'>]
Pos x: [<class 'str'>]
Pos y: [<class 'str'>]
Participantes: [<class 'str'>]
Víctima: [<class 'str'>]
Acusado: [<class 'str'>]


In [698]:
ver_nulos_por_colref(df_hechos, 'Cruce')[[ 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura', 'Cruce', 'Dirección normalizada']]

,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada


In [699]:
df_hechos.columns

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora entera', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura',
       'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x',
       'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

In [700]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     696 non-null    object        
 1   Cantidad víctimas      696 non-null    int64         
 2   Fecha                  696 non-null    datetime64[ns]
 3   Año                    696 non-null    int64         
 4   Mes                    696 non-null    int64         
 5   Día                    696 non-null    int64         
 6   Hora                   696 non-null    object        
 7   Hora entera            696 non-null    int64         
 8   Lugar del hecho        696 non-null    object        
 9   Tipo de calle          696 non-null    object        
 10  Calle                  696 non-null    object        
 11  Altura                 696 non-null    float64       
 12  Cruce                  696 non-null    object        
 13  Direc

In [701]:
detalleB_tipos_datos(df_hechos)

Id: [<class 'str'>]
 
 
 
 
 
Hora: [<class 'datetime.time'>]
 
Lugar del hecho: [<class 'str'>]
Tipo de calle: [<class 'str'>]
Calle: [<class 'str'>]
 
Cruce: [<class 'str'>]
Dirección normalizada: [<class 'str'>]
 
XY (CABA): [<class 'str'>]
Pos x: [<class 'str'>]
Pos y: [<class 'str'>]
Participantes: [<class 'str'>]
Víctima: [<class 'str'>]
Acusado: [<class 'str'>]


##  'XY (CABA)', 'Pos x'  'Pos y'



*Estas   3 columnas  de georefernciacion dode se muestra la latitud y la logitud que  permitiria ubicar en un plano el lugar preciso del accidente  y esto es de suma importacia, pero veo algunas celdas sin esta informacion y parece que solo tienen un punto `.`  esto lo pude ver cuando modificaba  la columna `Dirección normalizada`* 


    Altura  Cruce Dirección normalizada Comuna    XY (CABA) Pos x Pos y  \
38     NaN    NaN                   NaN     13  Point (. .)     .     .   
106    NaN    NaN                   NaN      4  Point (. .)     .     .   
119    NaN    NaN                   NaN      0  Point (. .)     .     .   

>El tipo   de datos es 'str' es su totalidad  vamos a analisar  lo siguiente:

1. revisar la cantidad de veces que aparece estos datos  `Point (. .)     .     . ` y donde
2. ver que relacion tiene con  otras columnas 
3. ver si pueden modificarse usando informacion de otras columnas 

>CONCLUCIONES

*Depues de realizar esos 3 puntos encontre 12 filas sin posibilidades de poder  imputar con valores razonables   tome la desicion de aliminarlas* 


In [702]:
df_hechos[['Pos x']]

,Pos x
0,-58.47533969
1,-58.50877521
2,-58.39040293
3,-58.46503904
4,-58.38718297
...,...
691,-58.43353773
692,-58.46739825
693,-58.47293407
694,-58.47066794


In [703]:
buscar_puntos(df_hechos)[['Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura',
       'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x',
       'Pos y']]

Filas y columnas que contienen un solo punto ('.'):
Fila: 38, Columna: Pos x
Fila: 38, Columna: Pos y
Fila: 106, Columna: Pos x
Fila: 106, Columna: Pos y
Fila: 119, Columna: Pos x
Fila: 119, Columna: Pos y
Fila: 139, Columna: Pos x
Fila: 139, Columna: Pos y
Fila: 176, Columna: Pos x
Fila: 176, Columna: Pos y
Fila: 180, Columna: Pos x
Fila: 180, Columna: Pos y
Fila: 181, Columna: Pos x
Fila: 181, Columna: Pos y
Fila: 256, Columna: Pos x
Fila: 256, Columna: Pos y
Fila: 313, Columna: Pos x
Fila: 313, Columna: Pos y
Fila: 546, Columna: Pos x
Fila: 546, Columna: Pos y
Fila: 559, Columna: Pos x
Fila: 559, Columna: Pos y
Fila: 621, Columna: Pos x
Fila: 621, Columna: Pos y


,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y
38,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0.0,SD,AUTOPISTA LUGONES PK 10000,13,Point (. .),.,.
106,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,0.0,SD,AU BUENOS AIRES - LA PLATA KM. 4,4,Point (. .),.,.
119,SD,CALLE,SD,0.0,SD,SD,0,Point (. .),.,.
139,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,0.0,SD,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.
176,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",0.0,SD,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,0.0,SD,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,9,Point (. .),.,.
181,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,0.0,SD,AU DELLEPIANE 2400,7,Point (. .),.,.
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,0.0,SD,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.
313,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0.0,SD,AUTOPISTA LUGONES KM 4.7,14,Point (. .),.,.
546,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",0.0,SD,"LUGONES, LEOPOLDO AV. KM 6,1",14,Point (. .),.,.


In [704]:
eliminar_filas_con_puntos_colref(df_hechos, 'Pos x')

[38, 106, 119, 139, 176, 180, 181, 256, 313, 546, 559, 621]

>Vemos que df_hechos se redujo en 12 filas 

In [705]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     684 non-null    object        
 1   Cantidad víctimas      684 non-null    int64         
 2   Fecha                  684 non-null    datetime64[ns]
 3   Año                    684 non-null    int64         
 4   Mes                    684 non-null    int64         
 5   Día                    684 non-null    int64         
 6   Hora                   684 non-null    object        
 7   Hora entera            684 non-null    int64         
 8   Lugar del hecho        684 non-null    object        
 9   Tipo de calle          684 non-null    object        
 10  Calle                  684 non-null    object        
 11  Altura                 684 non-null    float64       
 12  Cruce                  684 non-null    object        
 13  Dirección 

>Confirmamos y vemos que el indice 38 a sido eliminado solo por ver una de las filas eliminadas 

In [706]:
df_rango = df_hechos.iloc[35:40]

In [652]:
df_rango

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,...,0.0,SD,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
36,2016-0050,1,2016-04-19,2016,4,19,09:50:00,9,TIMOTEO GORDILLO Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,0.0,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","GORDILLO, TIMOTEO y FERNANDEZ DE LA CRUZ, F., ...",8,Point (99106.14718950 93761.32882893),-58.47305463,-34.68550712,PEATON-PASAJEROS,PEATON,PASAJEROS
37,2016-0051,1,2016-04-20,2016,4,20,04:00:00,4,AV AMANCIO ALCORTA Y BONAVENA,AVENIDA,...,0.0,"BONAVENA, OSCAR NATALIO","ALCORTA, AMANCIO AV. y BONAVENA, OSCAR NATALIO",4,Point (105230.98315750 97613.97258373),-58.40623949,-34.65076549,MOTO-CARGAS,MOTO,CARGAS
39,2016-0053,1,2016-04-22,2016,4,22,09:30:00,9,AV TRIUNVIRATO Y AV OLAZABAL,AVENIDA,...,0.0,OLAZABAL AV.,TRIUNVIRATO AV. y OLAZABAL AV.,12,Point (98073.80557770 105955.73972818),-58.48429180,-34.57558053,PEATON-PASAJEROS,PEATON,PASAJEROS
40,2016-0054,1,2016-04-23,2016,4,23,21:00:00,21,AU LUIS DELLEPIANE Y AV GRAL PAZ,AUTOPISTA,...,0.0,"PAZ, GRAL. AV.","AUTOPISTA DELLEPIANE LUIS TTE. GRAL. y PAZ, GR...",8,Point (97832.57143731 93489.83736533),-58.48695354,-34.68795245,MOTO-MOTO,MOTO,MOTO


# LIMITE DE CORRECION 
 hasta aqui esta garantizado el ETL bien ejecutado bajo mi criterio